# AutoCorrect

## Import Libraries

In [1]:
import numpy as np
import pickle
from collections import Counter

## Import vocab and Probability Dictionary
First we will import the vocabulary of our autocorrect along with the probability dictionary of the vocabulary.

In [2]:
with open("./vocab_prob_dicts.p", "rb") as f:
    prob_dict, vocab_word_count = pickle.load(f)

## Edit methods
Now, that we have probabilities for all the words in the corpus, It's time to writa a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words.
We will implement four functions:

- **delete_letter**: given a word, it returns all the possible strings that have one character removed.
- **switch_letter**: given a word, it returns all the possible strings that have two adjacent letters switched.
- **replace_letter**: given a word, it returns all the possible strings that have one character replaced by another different letter.
- **insert_letter**: given a word, it returns all the possible strings that have an additional character inserted.

### Delete Letter

In [3]:
def delete_letter(word, verbose = False):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    deletes = [L + R[1:] for L, R in splits if R]
    
    if verbose:
        print(f"Input word : {word}\nSplit list : {splits}\nDelete list : {deletes}")
    return deletes

In [4]:
delete_word_l = delete_letter(word="cans", verbose=True)

print(f"Number of outputs of delete_letter('at') is {len(delete_letter('at'))}")

Input word : cans
Split list : [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's'), ('cans', '')]
Delete list : ['ans', 'cns', 'cas', 'can']
Number of outputs of delete_letter('at') is 2


### Switch Letter

In [5]:
def switch_letter(word, verbose=False):
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    
    switches = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) >= 2]
    
    if verbose:
        if verbose: print(f"Input word = {word} \nsplit_l = {splits} \nswitch_l = {switches}") 

    return switches

In [6]:
switch_word_l = switch_letter("eta", verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


### Replace Letter

In [7]:
def replace_letter(word, verbose = False):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    string = "abcdefghijklmnopqrstuvwxyz'"
    replaced_l = []
    
    for L, R in splits:
        if R:
            for letter in string:
                replaced_l.append(L + letter + R[1:])
            
    replaced = set(replaced_l)
    replaced.discard(word)
    
    replaced_l = sorted(list(replaced))
    
    if verbose:
        print(f"Input word = {word} \nsplit_l = {splits} \nreplace_l {replaced_l}")
    
    return replaced_l

In [8]:
replace_l = replace_letter(word='can', verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n'), ('can', '')] 
replace_l ["'an", 'aan', 'ban', "c'n", "ca'", 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


### Insert Letter

In [9]:
def insert_letter(word, verbose = False):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    string = "abcdefghijklmnopqrstuvwxyz'"
    
    inserted_l = []
    
    for L, R in splits:
        for letter in string:
            inserted_l.append(L + letter + R)
                
    inserted = sorted(inserted_l)
    
    if verbose: print(f"Input word = {word} \nsplit_l = {splits} \nreplace_l {inserted_l}")
        
    return inserted

In [10]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word = at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
replace_l ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', "'at", 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', "a't", 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', "at'"]
Number of strings output by insert_letter('at') is 81


## Creating Edits
Now that we have our edit functions, we can put them all together to get a list of all the possible strings from the input string by applying the functions defines above. We will get the strings which are `ONE` or `TWO` edits away from being a correcct string.

### Edit One Letter

In [11]:
def edit_one_letters(word, allow_switches = True):
    edit_one_set = set()
    
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    
    return edit_one_set

In [12]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letters(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letters('at'))}")

input word at 
edit_one_l 
["'at", "'t", 'a', "a'", "a't", 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', "at'", 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

Number of outputs from edit_one_letter('at') is 134


### Edit Two Letters

In [13]:
def edit_two_letters(word, allow_switches = True):
    
    edit_two_set = set()
    
    edit_one = edit_one_letters(word, allow_switches=allow_switches)
    
    for w in edit_one:
        if w:
            edit_two = edit_one_letters(w, allow_switches=allow_switches)
            edit_two_set.update(edit_two)
            
    return edit_two_set

In [14]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2864
First 10 strings ['', "'", "''", "''a", "'a", "'a'", "'aa", "'ab", "'ac", "'ad"]
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
Number of strings that are 2 edit distances from 'at' is 7726


## Get Corrections
Now we are ready to get corrections for the input string. We follow the following `Greedy Algorithm` to get corrections.

1. If the word is in vocabulary, return the word (correct word)
2. Else get all strings that are one edit away<br>
    a. Check for strings in vocabulary<br>
    b. If found, return strings<br>
    c. Else Get all strings that are two edits away<br>
        i. Check for strings in vocabulary<br>
        ii. If found, return strings<br>
        iii. Else return the original string (no correcctions available)

The algorithm is an `Greedy Algorithm` as we are searching for strings that are at minimum edits from the input string at all time.

In [15]:
def get_corrections(word, probs, vocab, n = 2, verbose = False):
    suggestions = None
    n_best = []
    
    if(word in vocab):
        suggestions = [word]
    else:
        one_edit = edit_one_letters(word)
        in_one_edit = filter(lambda x: x in vocab, one_edit)
        if in_one_edit:
            suggestions = list(in_one_edit)
        else:
            two_edit = edit_two_letters(word)
            in_two_edit = filter(lambda x: x in vocab, two_edit)
            if in_two_edit:
                suggestions = list(in_two_edit)
            else:
                suggestions = [word]
                
    n_best = [[s, probs[s]] for s in suggestions]
    n_best = sorted(n_best, key=lambda x: x[1], reverse=True)       
    return n_best[:n]

## Running Corrections algorithm

In [16]:
my_word = 'hbi'
n_best = 3
vocab = list(vocab_word_count.keys())
corrections = get_corrections(my_word, prob_dict, vocab, n = n_best)

for i, word_probs in enumerate(corrections):
    print("Word : {}, probability : {:.6f}".format(word_probs[0], word_probs[1]))

Word : hai, probability : 0.012827
Word : bhi, probability : 0.003414
Word : hui, probability : 0.000785


# References 
- deeplearning.ai - https://www.deeplearning.ai/natural-language-processing-specialization/